In [143]:
import matplotlib.pyplot as plt
import numpy as np

In [144]:
np.set_printoptions(precision=8)

In [145]:
path="/Users/ecasiano/Desktop/PrototypeScripts/TestSystematicError/"
path="/Users/ecasiano/Xcode/pimc/pimc/"

In [170]:
seed = 42

In [173]:
data_SWAPn_m2 = np.loadtxt(path+"1D_4_4_4_3.300000_1.000000_2.000000_10000_10_SWAPn-mA2_"+str(seed)+"_square.dat")
data_PnSquared_m2 = np.loadtxt(path+"1D_4_4_4_3.300000_1.000000_2.000000_10000_10_PnSquared-mA2_"+str(seed)+"_square.dat")

data_SWAPn_m2.shape,data_PnSquared_m2.shape

((10, 5), (10, 5))

In [174]:
N=data_SWAPn_m2.shape[1]-1 # Get N

In [175]:
S2 = np.zeros(data_SWAPn_m2.shape)
SWAPn_m2_col_sum = np.sum(data_SWAPn_m2,axis=0)
PnSquared_m2_col_sum = np.sum(data_PnSquared_m2,axis=0)
for i in range(data_SWAPn_m2.shape[0]):
    PnSquared_m2_col_sum_i = PnSquared_m2_col_sum - data_PnSquared_m2[i]
    
    SWAPn_m2_col_sum_i = SWAPn_m2_col_sum - data_SWAPn_m2[i]
    
    S2[i] = -np.log(SWAPn_m2_col_sum_i/PnSquared_m2_col_sum_i)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [176]:
S2_mean = np.mean(S2,axis=0)
S2_mean

array([        nan, -0.03050586, -0.02384141, -0.08795588,         nan])

In [177]:
N_rows = S2.shape[0]

In [178]:
S2_err = np.std(S2,axis=0)
S2_err *= np.sqrt(N_rows-1)

In [179]:
for n in range(N+1):
    print(f"<S2(n={n})> = {S2_mean[n]:0.6f} +/- {S2_err[n]:0.6f}")

<S2(n=0)> = nan +/- nan
<S2(n=1)> = -0.030506 +/- 0.210126
<S2(n=2)> = -0.023841 +/- 0.033543
<S2(n=3)> = -0.087956 +/- 0.115582
<S2(n=4)> = nan +/- nan


### $P(n)$

In [131]:
data_Pn_m2 = np.loadtxt(path+"1D_4_4_4_3.300000_1.000000_2.000000_10000_1000_Pn-mA2_"+str(seed)+"_square.dat")
data_Pn_m2.shape

(156, 5)

In [116]:
data_Pn_col_sum_m2 = np.sum(data_Pn_m2,axis=0)

In [117]:
jacknifed_Pn = np.zeros(data_Pn_m2.shape)
for i in range(data_Pn_m2.shape[0]):
    data_Pn_col_sum_m2_i = data_Pn_col_sum_m2 - data_Pn_m2[i]
        
    jacknifed_Pn[i] = data_Pn_col_sum_m2_i
jacknifed_Pn /= np.sum(jacknifed_Pn,axis=1)[:,None]

In [118]:
Pn_mean = np.mean(jacknifed_Pn,axis=0)
Pn_mean /= np.sum(Pn_mean )
N_rows_Pn = jacknifed_Pn.shape[0]
Pn_err = np.std(jacknifed_Pn,axis=0)
Pn_err *= np.sqrt(N_rows_Pn-1)

In [119]:
for n in range(N+1):
    print(f'P(n={n}) = {Pn_mean[n]:0.4f} +/- {Pn_err[n]:0.4f}')

P(n=0) = 0.0090 +/- 0.0005
P(n=1) = 0.1942 +/- 0.0020
P(n=2) = 0.5884 +/- 0.0023
P(n=3) = 0.1979 +/- 0.0022
P(n=4) = 0.0105 +/- 0.0006
